## Training Notebook

This notebook illustrates training of a simple model to classify digits using the MNIST dataset. This code is used to train the model included with the templates. This is meant to be a starter model to show you how to set up Serverless applications to do inferences. For deeper understanding of how to train a good model for MNIST, we recommend literature from the [MNIST website](http://yann.lecun.com/exdb/mnist/). The dataset is made available under a [Creative Commons Attribution-Share Alike 3.0](https://creativecommons.org/licenses/by-sa/3.0/) license.

## Tensorflow Model Training

For this example, we will train a simple CNN classifier using Tensorflow to classify the MNIST digits. We will then freeze the model in the `.h5` format. This is same as the starter model file included with the SAM templates.

In [103]:
import tensorflow as tf
from keras import backend as K
import tifffile as tiff
import numpy as np
import pandas as pd
import cv2
from shapely.geometry import MultiPolygon, Polygon
from shapely.wkt import loads as wkt_loads
import shapely.wkt
import shapely.affinity
from collections import defaultdict


size = 160
s = 835
smooth = 1e-12

def mask_to_polygons(mask, epsilon=5, min_area=1.):
    """
    converts a mask into polygons.
    """
    
    contours, hierarchy = cv2.findContours(((mask == 1) * 255).astype(np.uint8), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()

    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1

    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])

    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

def adjust_contrast(bands, lower_percent=2, higher_percent=98):
    """
    to adjust the contrast of the image 
    bands is the image 
    """
    out = np.zeros_like(bands).astype(np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)

#def M(image_id):
#    # __author__ = amaia
#    # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
#    zip_path = 'sixteen_band.zip'
#    tgtImg = '{}_M.tif'.format(image_id)
#    with zipfile.ZipFile(zip_path) as myzip:
#        files_in_zip = myzip.namelist()
#        for fname in files_in_zip:
#            if fname.endswith(tgtImg):
#                with myzip.open(fname) as myfile:
#                    img = tiff.imread(myfile)
#                    img = np.rollaxis(img, 0, 3)
#                    return img
def M(img_pat):
    with open("vlj_img_1029.tiff", 'rb') as myfile:
        img = tiff.imread(myfile)
        img = np.rollaxis(img, 0, 3)
        return img


class Dataset:
  
    def __init__(self, images_dir, mask_dir):
        
        self.ids = images_dir
        self.images_fps = images_dir
        self.masks_fps  = mask_dir
    
    def __getitem__(self, i):
        
        # read data
        image = np.load(self.images_fps[i]) 
        mask  = np.load(self.masks_fps[i])

          
        image = np.stack(image, axis=-1).astype('float')
        mask = np.stack(mask, axis=-1).astype('float')

        #image = np.transpose(image, (1,0,2)) 
        #mask = np.transpose(mask, (1,0,2)) 
    
        image = np.transpose(image, (0,2,1)) 
        mask = np.transpose(mask, (0,2,1)) 
  
        return image, mask
      
    def __len__(self):
        return len(self.ids)

class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        #print(len(batch))
        return tuple(batch)
    
    def __len__(self):
        return len(self.indexes) // self.batch_size


def jaccard_coef(y_true, y_pred):
    """
    Jaccard Index: Intersection over Union.
    J(A,B) = |A∩B| / |A∪B| 
         = |A∩B| / |A|+|B|-|A∩B|
    """
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    total = K.sum(y_true + y_pred, axis=[0, -1, -2])
    union = total - intersection

    jac = (intersection + smooth) / (union+ smooth)

    return K.mean(jac)

def SegNet():
    
    tf.random.set_seed(32)
    classes= 10
    img_input = tf.keras.layers.Input(shape=(size, size, 8))
    x = img_input

    # Encoder 
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 23))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',  kernel_initializer = tf.keras.initializers.he_normal(seed= 43))(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 32))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 41))(x)

    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 33))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 35))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 54))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 39))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    #Decoder
    
    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 45))(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 41))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 49))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
      
    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 18))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 21))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(classes, kernel_size=3, activation='relu', padding='same', kernel_initializer = tf.keras.initializers.he_normal(seed= 16))(x)
    x = tf.keras.layers.Dropout(0.25)(x)
  
    x = tf.keras.layers.Activation("softmax")(x)
    
    model = tf.keras.Model(img_input, x)
  
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),loss='binary_crossentropy', metrics=[jaccard_coef])
    return model

In [104]:
model = SegNet()
model.load_weights("model-weights.hdf5")
img = adjust_contrast(M(1)).copy()
res = model.predict(img.reshape((1, 160, 160, 8)))
threshold = 0.5
pred_binary_mask = res >= threshold


# Get all the predicted classes into a dataframe
DF  = pd.DataFrame(columns=["image", "class", "poly", 'Multi'])
image, cl , ploy, t_l = [],[],[], []
i = 0
for j in range(10):
    ab = mask_to_polygons(pred_binary_mask[0, :,:,j], epsilon=1)
    t = shapely.wkt.dumps(ab)
    t_l.append(t)
    image.append(i+1)
    cl.append(j+1)
    ploy.append(len(ab))
    df = pd.DataFrame(list(zip(image, cl, ploy, t_l)), columns = ['image', 'class', 'poly', 'Multi'])
DF = pd.concat([DF,df], ignore_index=True)

In [105]:
DF

,image,class,poly,Multi
0,1,1,0,GEOMETRYCOLLECTION EMPTY
1,1,2,0,GEOMETRYCOLLECTION EMPTY
2,1,3,0,GEOMETRYCOLLECTION EMPTY
3,1,4,1,MULTIPOLYGON (((119.0000000000000000 155.00000...
4,1,5,65,MULTIPOLYGON (((129.0000000000000000 154.00000...
5,1,6,8,MULTIPOLYGON (((133.0000000000000000 1.0000000...
6,1,7,0,GEOMETRYCOLLECTION EMPTY
7,1,8,0,GEOMETRYCOLLECTION EMPTY
8,1,9,0,GEOMETRYCOLLECTION EMPTY
9,1,10,0,GEOMETRYCOLLECTION EMPTY


In [96]:
DF

,image,class,poly,Multi
0,1,1,0,GEOMETRYCOLLECTION EMPTY
1,1,2,0,GEOMETRYCOLLECTION EMPTY
2,1,3,0,GEOMETRYCOLLECTION EMPTY
3,1,4,1,MULTIPOLYGON (((119.0000000000000000 155.00000...
4,1,5,65,MULTIPOLYGON (((129.0000000000000000 154.00000...
5,1,6,8,MULTIPOLYGON (((133.0000000000000000 1.0000000...
6,1,7,0,GEOMETRYCOLLECTION EMPTY
7,1,8,0,GEOMETRYCOLLECTION EMPTY
8,1,9,0,GEOMETRYCOLLECTION EMPTY
9,1,10,0,GEOMETRYCOLLECTION EMPTY


In [28]:
import requests
import base64

with open("vlj_img_1029.tiff", "rb") as image_file:
    #base64.b64encode(image_file.read()).decode('utf-8')
    data = image_file.read()
    r = requests.post("https://hxy1cn1sl8.execute-api.us-east-1.amazonaws.com/Prod/segment_tiff", data=data)
    print(r.status_code)

502


In [30]:
r

<Response [502]>

In [16]:
image_file

<_io.BufferedReader name='vlj_img_1029.tiff'>